In [ ]:
import torch, torchvision
import mmseg
from mmseg.registry import DATASETS
from mmseg.datasets import *
import mmcv
import mmengine
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os.path as osp
import numpy as np
from PIL import Image
from mmengine.runner import Runner
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
%cd mmsegmentation

root = '/workspaces/ECE661GroupProject_TransferLearning/data'
img_dir = 'image_png_256_gray'
ann_dir = 'fluid_png_256_binary'

classes = ('image', 'fluid')

In [ ]:
@DATASETS.register_module()
class BOE_Chiu_Dataset(BaseSegDataset):
    METAINFO = dict(classes = classes)
    def __init__(self,dataset=None, times=None, **kwargs):
        super(BOE_Chiu_Dataset, self).__init__(img_suffix='.png', seg_map_suffix='.png', **kwargs)

In [ ]:
cfg = mmengine.Config.fromfile('/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/unet-s5-d16_deeplabv3_4xb4-40k_hrf-256x256.py')

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type="BN", requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

# Modify dataset type and path
cfg.dataset_type = "BOE_Chiu_Dataset"
cfg.data_root = root

cfg.train_dataloader.batch_size = 2

cfg.train_pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="LoadAnnotations"),
    dict(type="Resize", scale=(512, 512), keep_ratio=True),
    dict(type="PackSegInputs"),
]

cfg.test_pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="Resize", scale=(512, 512), keep_ratio=True),
    dict(type="LoadAnnotations"),
    dict(type="PackSegInputs"),
]

# Configure train dataloader
cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(
    img_path=img_dir, seg_map_path=ann_dir
)
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = "splits/train.txt"

# Configure val & test dataloader
cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(
    img_path=img_dir, seg_map_path=ann_dir
)
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = "splits/val.txt"

cfg.test_dataloader = cfg.val_dataloader

# Configure Evaluator
cfg.test_evaluator = ["mDice", "mIoU", "mFscore"]
cfg.val_evaluator.iou_metrics = ["mDice", "mIoU", "mFscore"]

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/deeplabv3_unet_s5-d16_256x256_40k_hrf_20201226_094047-3a1fdf85.pth'

# Set up working dir to save files and logs.
cfg.work_dir = "./work_dirs/tutorial"

cfg.train_cfg.max_iters = 200
cfg.train_cfg.val_interval = 200
cfg.default_hooks.logger.interval = 10
cfg.default_hooks.checkpoint.interval = 200

# Set seed to facitate reproducing the result
cfg.randomness = dict(seed=0)

# Let's have a look at the final config used for training
print(f"Config:\n{cfg.pretty_text}")

runner = Runner.from_cfg(cfg)

runner.train()